In [2]:
import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal 

## App1 - Load multiple file & Output CV, GV, GP/W

In [2]:
### new light sweep dataset ###

# Testing one csv file
df1 = pd.read_csv("../Requirements/Final Datasets (ALL Tasks)/MFCV-1  Light #1-2_light.csv", encoding='utf-8', sep='\t')
df1 = df1.iloc[:, 0].str.split(',', expand=True)

# auto retrieve VBias (if dataset change) (*** data can used in MFCV all data (light, low, high ....) ***)
a = df1.iloc[:, 0].str.strip().tolist()
indexes = [index for index in range(len(a)) if a[index] == 'DataName']
if len(indexes) == 0 :
    indexes = [index for index in range(len(a)) if a[index] == 'VBias']
    if len(indexes) == 0:
        a = df1.iloc[:, 1].str.strip().tolist()
        indexes = [index for index in range(len(a)) if a[index] == 'VBias']
        df1 = df1.iloc[:, 1:][indexes[0]:].dropna(axis=1)
    else:
         df1 = df1.iloc[:, 0:][indexes[0]:].dropna(axis=1)
else:
    df1 = df1.iloc[:, 1:][indexes[0]:].dropna(axis=1)

# Filter datasets
df1 = df1.rename(columns=df1.iloc[0]).drop(df1.index[0]).reset_index(drop=True)
df1.columns = df1.columns.str.strip() # To strip whitespaces
df1 = df1.loc[:,["VBias","Freq", "C", "G"]].apply(pd.to_numeric,1)
df1


,VBias,Freq,C,G
0,-10.0,1000.0,1.196770e-10,3.010200e-09
1,-9.8,1000.0,1.197770e-10,2.869730e-09
2,-9.6,1000.0,1.198500e-10,2.832180e-09
3,-9.4,1000.0,1.199060e-10,2.698470e-09
4,-9.2,1000.0,1.199520e-10,2.811320e-09
...,...,...,...,...
2015,9.2,13600.5,1.350320e-10,5.670050e-08
2016,9.4,13600.5,1.350330e-10,5.656140e-08
2017,9.6,13600.5,1.350440e-10,5.673630e-08
2018,9.8,13600.5,1.350690e-10,5.722300e-08


In [3]:
# Testing multiple CSV 
excelPaths = ["../Requirements/Final Datasets (ALL Tasks)/MFCV-1  Light #1-2_light.csv", 
              "../Requirements/Final Datasets (ALL Tasks)/MFCV-2  Light #1-2_light.csv", 
              "../Requirements/Final Datasets (ALL Tasks)/MFCV-3  Light #1-2_light.csv", 
              "../Requirements/Final Datasets (ALL Tasks)/MFCV-4  Light #1-2_light.csv"]

dfs = []
_Freq = []
_W = []
_V = []
_Area = 1 * 1e-3
for i in range(len(excelPaths)):
            dfs.append("df" + str(i))
            dfs[i] = pd.DataFrame()
            dfs[i] = pd.read_csv(excelPaths[i], encoding='utf-8', sep='\t')
            dfs[i] = dfs[i].iloc[:, 0].str.split(',', expand=True)
            a = dfs[i].iloc[:, 0].str.strip().tolist()
            indexes = [index for index in range(len(a)) if a[index] == 'DataName']
            if len(indexes) == 0:
                indexes = [index for index in range(len(a)) if a[index] == 'VBias']
                if len(indexes) == 0:
                    a = dfs[i].iloc[:, 1].str.strip().tolist()
                    indexes = [index for index in range(len(a)) if a[index] == 'VBias']
                    dfs[i] = dfs[i].iloc[:, 1:][indexes[0]:].dropna(axis=1)
                else:
                    dfs[i] = dfs[i].iloc[:, 0:][indexes[0]:].dropna(axis=1)
            else:
                dfs[i] = dfs[i].iloc[:, 1:][indexes[0]:].dropna(axis=1)
            dfs[i] = dfs[i].rename(columns=dfs[i].iloc[0]).drop(dfs[i].index[0]).reset_index(drop=True)
            dfs[i].columns = dfs[i].columns.str.strip() # To strip whitespaces
            dfs[i] = dfs[i].loc[:,["VBias","Freq", "C", "G"]].apply(pd.to_numeric,1)
            
            # Definite Value
            _Freq += sorted(list(set(dfs[i]["Freq"].tolist())))
            _V = dfs[0].query("Freq == @_Freq[0]")["VBias"].tolist()  # all V #df1.iloc[2:154, 0].tolist()  # 152 (-15 ~ 15)
            _Area = 1 * 1e-3

            
G1 = pd.DataFrame(columns = _Freq) # C
G2 = pd.DataFrame(columns = _Freq) # G

for i in range(len(_Freq)):
    _W.append(2 * (math.pi) * _Freq[i])
    listC = []
    listG = []
    for j in range(len(excelPaths)):
        listC += dfs[j].query("Freq == @_Freq[@i]")["C"].tolist()
        listG += dfs[j].query("Freq == @_Freq[@i]")["G"].tolist()

    G1[_Freq[i]] = listC
    G2[_Freq[i]] = listG
    
G1           

,1000.0,1147.3,1316.2,1500.0,1740.0,1987.5,2290.0,2615.9,3010.0,3443.1,...,1452188.0,1666033.0,1911368.0,2200000.0,2530000.0,2886202.0,3311217.0,3798817.0,4358220.0,5000000.0
0,1.196770e-10,1.196710e-10,1.196650e-10,1.196460e-10,1.196450e-10,1.196250e-10,1.196070e-10,1.196190e-10,1.195870e-10,1.195910e-10,...,1.128800e-10,1.097820e-10,1.062240e-10,1.006090e-10,9.304820e-11,8.321400e-11,6.781780e-11,4.471310e-11,8.399130e-12,-5.726630e-11
1,1.197770e-10,1.197450e-10,1.197620e-10,1.197250e-10,1.197350e-10,1.197210e-10,1.197030e-10,1.196760e-10,1.196850e-10,1.196660e-10,...,1.128830e-10,1.099720e-10,1.062580e-10,1.007280e-10,9.315130e-11,8.313290e-11,6.766610e-11,4.475750e-11,8.352110e-12,-5.729700e-11
2,1.198500e-10,1.198120e-10,1.198060e-10,1.197980e-10,1.198070e-10,1.197690e-10,1.197850e-10,1.197680e-10,1.197330e-10,1.197320e-10,...,1.129250e-10,1.099610e-10,1.062740e-10,1.006880e-10,9.295750e-11,8.317690e-11,6.743220e-11,4.467470e-11,8.326840e-12,-5.732010e-11
3,1.199060e-10,1.198880e-10,1.198780e-10,1.198550e-10,1.198420e-10,1.198250e-10,1.198050e-10,1.198010e-10,1.198060e-10,1.197770e-10,...,1.129800e-10,1.100680e-10,1.062810e-10,1.006160e-10,9.307940e-11,8.289700e-11,6.754110e-11,4.461720e-11,8.170770e-12,-5.736970e-11
4,1.199520e-10,1.199240e-10,1.199240e-10,1.199120e-10,1.198960e-10,1.198900e-10,1.198470e-10,1.198610e-10,1.198420e-10,1.198390e-10,...,1.129830e-10,1.100550e-10,1.063390e-10,1.006990e-10,9.318750e-11,8.295130e-11,6.755130e-11,4.466340e-11,8.088980e-12,-5.746460e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.350300e-10,1.350290e-10,1.350200e-10,1.350370e-10,1.350410e-10,1.350360e-10,1.350320e-10,1.350390e-10,1.350440e-10,1.350430e-10,...,1.195080e-10,1.143950e-10,1.077990e-10,9.906110e-11,8.798520e-11,7.416540e-11,5.639310e-11,3.159610e-11,-5.150140e-12,-6.819520e-11
97,1.350550e-10,1.350390e-10,1.350590e-10,1.350520e-10,1.350830e-10,1.350770e-10,1.350400e-10,1.350380e-10,1.350390e-10,1.350570e-10,...,1.195380e-10,1.143660e-10,1.077980e-10,9.922010e-11,8.772810e-11,7.403600e-11,5.645300e-11,3.125050e-11,-5.195060e-12,-6.819910e-11
98,1.350820e-10,1.350670e-10,1.350760e-10,1.350720e-10,1.350700e-10,1.350600e-10,1.350610e-10,1.350740e-10,1.350600e-10,1.350700e-10,...,1.195230e-10,1.144740e-10,1.079580e-10,9.930040e-11,8.752720e-11,7.415480e-11,5.622560e-11,3.147580e-11,-5.302110e-12,-6.827680e-11
99,1.350800e-10,1.350760e-10,1.350790e-10,1.350830e-10,1.350860e-10,1.350790e-10,1.350690e-10,1.350790e-10,1.350880e-10,1.350750e-10,...,1.194430e-10,1.145530e-10,1.079060e-10,9.929910e-11,8.787950e-11,7.408330e-11,5.623510e-11,3.137900e-11,-5.250370e-12,-6.827780e-11


## App 2 - 

In [79]:
# Gp_W_Results(with_Noise)
df1 = pd.read_excel('./Gp_W_Results(with_Noise).xlsx' , index_col=0)
df1

,-10.0,-9.8,-9.6,-9.4,-9.2,-9.0,-8.8,-8.6,-8.4,-8.2,...,8.2,8.4,8.6,8.8,9.0,9.2,9.4,9.6,9.8,10.0
W,,,,,,,,,,,,,,,,,,,,,
6.283185e+03,3.670881e-08,3.545666e-08,3.532896e-08,3.391209e-08,3.554253e-08,3.361912e-08,3.261360e-08,3.215306e-08,3.384250e-08,3.468341e-08,...,5.430567e-05,6.493310e-05,7.326388e-05,7.687676e-05,8.497138e-05,7.670853e-05,8.233467e-05,8.585297e-05,8.285903e-05,1.045952e-04
7.208699e+03,1.891692e-08,2.044934e-08,1.937970e-08,1.871670e-08,1.944836e-08,2.069898e-08,1.996870e-08,1.950038e-08,1.924375e-08,2.009429e-08,...,6.575896e-05,7.946080e-05,8.604746e-05,1.040308e-04,1.326913e-04,1.495710e-04,1.557636e-04,1.896227e-04,1.979004e-04,2.202012e-04
8.269929e+03,1.087732e-08,9.227300e-09,8.891896e-09,9.692691e-09,7.333783e-09,1.115260e-08,9.271948e-09,9.996373e-09,8.537110e-09,1.145898e-08,...,7.052310e-05,9.206239e-05,8.961751e-05,1.073425e-04,1.552092e-04,1.372104e-04,1.825676e-04,2.005161e-04,1.669562e-04,2.039786e-04
9.424778e+03,9.590248e-09,7.971733e-09,1.067895e-08,9.836772e-09,8.671361e-09,1.016939e-08,9.812814e-09,9.769587e-09,9.256033e-09,9.046167e-09,...,5.765646e-05,6.763848e-05,9.604179e-05,1.125575e-04,1.098605e-04,1.434975e-04,1.551345e-04,2.232796e-04,2.347867e-04,2.093947e-04
1.093274e+04,8.941987e-09,1.090846e-08,1.016342e-08,8.413564e-09,1.040616e-08,1.042788e-08,1.033380e-08,9.296425e-09,9.329923e-09,1.088254e-08,...,6.206980e-05,8.144946e-05,9.524726e-05,9.034518e-05,1.135587e-04,1.381723e-04,1.628755e-04,1.573570e-04,1.517102e-04,1.653258e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.813454e+07,5.692303e-08,5.687144e-08,5.688795e-08,5.685486e-08,5.684494e-08,5.676247e-08,5.682028e-08,5.674552e-08,5.672041e-08,5.671889e-08,...,5.186655e-08,5.183849e-08,5.185303e-08,5.181856e-08,5.183479e-08,5.177402e-08,5.179288e-08,5.176136e-08,5.176650e-08,5.174226e-08
2.080499e+07,3.238493e-08,3.240047e-08,3.241695e-08,3.238634e-08,3.237156e-08,3.235153e-08,3.236360e-08,3.232585e-08,3.235283e-08,3.238935e-08,...,3.050610e-08,3.047838e-08,3.045219e-08,3.044880e-08,3.043587e-08,3.041143e-08,3.039837e-08,3.043358e-08,3.041484e-08,3.041968e-08
2.386867e+07,1.209515e-08,1.208747e-08,1.207174e-08,1.208106e-08,1.207533e-08,1.207215e-08,1.205320e-08,1.205254e-08,1.206316e-08,1.205487e-08,...,1.175484e-08,1.175857e-08,1.176306e-08,1.174487e-08,1.176552e-08,1.174197e-08,1.176251e-08,1.173557e-08,1.173943e-08,1.174510e-08


In [94]:
_W = df1.index.tolist()
_V = list(map(float, df1.columns.tolist()))

In [95]:
df2 = df1.to_numpy()
_Gdf2[1,0]

1.891691673448087e-08